In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
#i need that data that contaons just 3's and 7's
#that is my first step
#importing all i need to make my life easy


#hide
from fastai.vision.all import *
#from fastbook import *

matplotlib.rc('image', cmap='Greys')

In [3]:
path = untar_data(URLs.MNIST_SAMPLE)


In [4]:
path.ls()

(#3) [Path('/root/.fastai/data/mnist_sample/valid'),Path('/root/.fastai/data/mnist_sample/labels.csv'),Path('/root/.fastai/data/mnist_sample/train')]

In [5]:
(path/'train').ls()


(#2) [Path('/root/.fastai/data/mnist_sample/train/7'),Path('/root/.fastai/data/mnist_sample/train/3')]

In [6]:
threes = (path/'train'/'3').ls().sorted()
sevens = (path/'train'/'7').ls().sorted()


In [7]:
seven_tensors = [tensor(Image.open(o)) for o in sevens]
three_tensors = [tensor(Image.open(o)) for o in threes]
len(three_tensors),len(seven_tensors)

(6131, 6265)

In [8]:
stacked_threes = torch.stack(three_tensors).float()/255
stacked_sevens = torch.stack(seven_tensors).float()/255

In [9]:
train_x = torch.cat([stacked_threes, stacked_sevens]).view(-1, 28*28)
train_x.shape


torch.Size([12396, 784])

In [10]:
train_y = tensor([1]*len(stacked_threes) + [0]*len(stacked_sevens)).unsqueeze(1)
train_y.shape

torch.Size([12396, 1])

In [11]:
ds = list(zip(train_x , train_y))
x, y = ds[0]
x.shape, y 

(torch.Size([784]), tensor([1]))

In [12]:
dl = DataLoader(ds, batch_size = 256)
xb, yb = first(dl)
xb.shape , yb.shape

(torch.Size([256, 784]), torch.Size([256, 1]))

In [13]:
valid_3_tens = torch.stack([tensor(Image.open(o))
                            for o in (path/'valid'/'3').ls()])
valid_3_tens = valid_3_tens.float()/255
valid_7_tens = torch.stack([tensor(Image.open(o))
                            for o in (path/'valid'/'7').ls()])
valid_7_tens = valid_7_tens.float()/255
valid_3_tens.shape,valid_7_tens.shape

(torch.Size([1010, 28, 28]), torch.Size([1028, 28, 28]))

In [14]:
valid_x = torch.cat([valid_3_tens, valid_7_tens]).view(-1, 28*28)
valid_y = tensor([1]*len(valid_3_tens) + [0]*len(valid_7_tens)).unsqueeze(1)
valid_dset = list(zip(valid_x,valid_y))

In [15]:
valid_dl = DataLoader(valid_dset, batch_size=256)

__STEP 1 : INITIALIZE PARAMS__

In [16]:
def init_params(size, std=1.0):
    return torch.randn(size).requires_grad_()

In [17]:
weights = init_params(28*28,1)
bias =init_params(1)

#STEP 2 : PREDICT FUNCTION

In [18]:
def linear1(xb):
    preds = xb@weights + bias
    return preds 

__STEP 3 : LOSS FUNCTION__

In [19]:
def mnist_loss(preds, targets):
    preds = preds.sigmoid()
    return torch.where(targets==1.0, 1-preds, preds).mean()
    

__STEP 4 : CALCULATE GRADIENTS__

In [20]:
def calc_grad(xb, yb, model):
    preds = model(xb)
    loss= mnist_loss(preds, yb)
    loss.backward()

__STEP 5 : BATCH ACCURACY FUNCTION__

In [21]:
def batch_accuracy(xb, yb):
    preds = xb.sigmoid()
    corrects = (preds > 0.5).float() == yb
    return corrects.float().mean()
    

__STEP 6 : VALIDATION_ACCURACY__

In [22]:
def validation_accuracy(model):
    accs = [batch_accuracy(model(xb),yb) for xb , yb in valid_dl]
    return round(torch.stack(accs).mean().item(), 4)

__STEP 7 : TRAIN EPOCH__

In [23]:
def train_epoch(params , model, lr):
    for xb , yb in dl:
        calc_grad(xb, yb ,model)
        for p in params:
            p.data -= p.grad*lr
            p.grad.zero_()
            

In [24]:
params = weights, bias
lr = 1.0
train_epoch(params, linear1, lr)
validation_accuracy(linear1)

0.7166

In [25]:
for i in range(20):
    train_epoch(params, linear1, lr)
    print(validation_accuracy(linear1), end=' ')

0.8639 0.9064 0.9246 0.9394 0.9442 0.9486 0.9528 0.9552 0.9572 0.9581 0.9585 0.9589 0.9594 0.9599 0.9604 0.9623 0.9637 0.9637 0.9637 0.9647 

__STEP 8 : BASIC OPTIMIZATION__

#pytorch creates a module to create a linear module  and parameters all at once 
#so we are gonna need that for our basic optimization

In [26]:
model = nn.Linear(28*28, 1) #creating an object from module

weight, bias= model.parameters()
weights.shape, bias.shape

(torch.Size([784]), torch.Size([1]))

In [27]:
class BasicOptim:
    def __init__(self, params, lr):
        self.params = list(params)
        self.lr = lr 

    def step(self):
        for p in self.params:
            p.data -= p.grad * lr

    def zero_grad(self):
        for p in self.params:
            p.grad.zero_()

In [28]:
opt = BasicOptim(model.parameters(), lr=lr)

In [29]:
def train_epoch(model):
    for xb , yb in dl:
        calc_grad(xb, yb ,model)
        opt.step()
        opt.zero_grad()

In [30]:
train_epoch(model)
validation_accuracy(model)

0.4932

__STEP 8 : SGD FROM FASTAI__

In [31]:
model = nn.Linear(28*28, 1) #creating an object from module

weight, bias= model.parameters()
weights.shape, bias.shape

opt = SGD(model.parameters(), lr=lr)

In [32]:
def train_epoch(model):
    for xb , yb in dl:
        calc_grad(xb, yb ,model)
        opt.step()
        opt.zero_grad()

In [33]:
train_epoch(model)
validation_accuracy(model)

0.4932

__STEP 9: TRAIN A MODEL__


In [34]:
def train_model(model, epoch):
    for i in range(epoch):
        train_epoch(model)
        print(validation_accuracy(model), end= ' ')

In [35]:
train_model(model, 20)

0.7612 0.8603 0.917 0.9345 0.9492 0.956 0.9628 0.9658 0.9667 0.9697 0.9721 0.9736 0.975 0.976 0.977 0.9775 0.978 0.978 0.9785 0.9789 

#fastai provides a more decent way of doing this. 

In [36]:
dls = DataLoaders(dl , valid_dl)

In [37]:
model = nn.Linear(28*28, 1)

In [38]:
learn = Learner( dls, model, loss_func= mnist_loss, opt_func = SGD, metrics = batch_accuracy)

In [39]:
learn.fit(10, lr=lr)

epoch,train_loss,valid_loss,batch_accuracy,time
0,0.636951,0.503248,0.495584,00:00
1,0.453753,0.233757,0.788518,00:00
2,0.169376,0.164225,0.852306,00:00
3,0.075783,0.101148,0.915604,00:00
4,0.041184,0.075196,0.934249,00:00
5,0.027598,0.060749,0.948970,00:00
6,0.021970,0.051626,0.957311,00:00
7,0.019432,0.045524,0.962709,00:00
8,0.018119,0.041226,0.965653,00:00
9,0.017314,0.038054,0.968106,00:00


__STEP 9 : A SIMPLE NEURAL NETWORK FUNCTION__

In [40]:
simple_net = nn.Sequential(
    nn.Linear(28*28, 30),
    nn.ReLU(),
    nn.Linear(30, 1)
)

In [41]:
learn = Learner(dls, simple_net, loss_func= mnist_loss, opt_func=SGD, metrics = batch_accuracy)

In [42]:
learn.fit(40, 0.1)

epoch,train_loss,valid_loss,batch_accuracy,time
0,0.310558,0.410624,0.506379,00:00
1,0.144614,0.227915,0.808636,00:00
2,0.080026,0.114803,0.915113,00:00
3,0.052637,0.077529,0.942591,00:00
4,0.039957,0.060574,0.956330,00:00
5,0.033486,0.051077,0.964181,00:00
6,0.029774,0.045096,0.967125,00:00
7,0.027365,0.041007,0.969087,00:00
8,0.025633,0.038035,0.968106,00:00
9,0.024294,0.035767,0.970559,00:00
